# Assignment 2: Card Draw

![Assignment 2 Image of Five Playing Cards](img/Card_Draw.png)

Author: Irene Kilgannon

Write a program that 'deals' i.e. prints out 5 cards from this API, https://deckofcardsapi.com/.

First shuffle the deck and get the deck_id. With the deck_id, get the cards. Print the value and suit of each card.

Extra marks: If the user has drawn a pair, triple, straight, or all of the same suit and congratulate the user.

In [42]:
# Import modules
import requests
import json
import pandas as pd
from emoji import emojize

## Shuffle The 'Cards'

In [43]:
shuffle_cards_url = "https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1"

# Get data from the API
response = requests.get(shuffle_cards_url)

# Convert JSON response to Python dictionary
response_dict = response.json()

# View keys in response_dict
#for key in response_dict:
#    print(key, ":", response_dict[key])

print('Deck ID:', response_dict['deck_id'])

Deck ID: qxb9y6lten0l


In [44]:
# Create a variable of the deck_id
deck_id =  response_dict['deck_id']

## Deal Five Cards

In [45]:
deal_cards_url = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=5"

# Get data from the API
response2 = requests.get(deal_cards_url)

# Convert JSON response to Python dictionary
deal_cards = response2.json()

add_card = []

for card in deal_cards['cards']:
    add_card.append(card['value'])
    print(f"{card['value']} of {card['suit']}")

KING of CLUBS
3 of HEARTS
4 of HEARTS
6 of DIAMONDS
JACK of HEARTS


In [46]:
add_card

['KING', '3', '4', '6', 'JACK']

In [47]:
# Write data to a JSON file
with open ("data/deal_cards.json", "w") as fp:
    json.dump(deal_cards, fp)

## Extra Marks

Extra marks: If the user has drawn a pair, triple, straight, or all of the same suit and congratulate the user.

Pair or triple - two or three cards of same value.

Straight - five cards of sequential rank, not all the same suit.

Suit - all cards in came suit.

In [48]:
# Convert to a DataFrame
df = pd.DataFrame(deal_cards['cards'], columns= ['value', 'suit'])
df

,value,suit
0,KING,CLUBS
1,3,HEARTS
2,4,HEARTS
3,6,DIAMONDS
4,JACK,HEARTS


In [49]:
print(f'There are {df['value'].duplicated().sum()} duplicated rows in the value column.')

There are 0 duplicated rows in the value column.


In [50]:
# View the duplicate rows.
df2 = df[df['value'].duplicated(keep = False)]
df2

,value,suit


In [51]:
# Checking for a pair.
if df['value'].duplicated().sum() == 1:
    print(f"{emojize(":partying_face:")} Congratulations!!! You have a pair of {df2.iloc[0, 0]}'s.")

# Check for 2 pairs or a triple
elif df['value'].duplicated().sum() == 2:
    # Check for a triple
    if len(df2) == 2:
        print(f"{emojize(":party_popper:")} Congratulations!!! You have a triple of {df2.iloc[0, 0]}'s.")
    # Check for 2 pairs
    else:
        print(f"{emojize(":party_popper:")} WOW! Congratulations!! You have a two pairs {df2.iloc[0, 0]} and {df2.iloc[0, 2]}.")
else:
    print(f'{emojize(":disappointed_face:")} Unlucky, no pairs or triples. Better luck next time.')

😞 Unlucky, no pairs or triples. Better luck next time.


In [52]:
# Check if all cards have the same suit.
if df['suit'].duplicated().sum() == 4:
    print(f'{emojize(":fireworks:")} All cards have the same suit. Congratulations! {emojize(":fireworks:")}')

In [53]:
# For straight need list of card values
card_values = ['ACE', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'JACK', 'QUEEN', 'KING']


#for card in card_values:
    #print(card)    

In [54]:
add_card

['KING', '3', '4', '6', 'JACK']

need to check if the 5 values in the add_card are in the order of card_values. 

In [55]:
# Sort value column in a specific order, https://stackoverflow.com/questions/52784410/sort-column-in-pandas-dataframe-by-specific-order
df.value = pd.Categorical(df.value, categories= card_values)

df = df.sort_values(by ='value')
df

,value,suit
1,3,HEARTS
2,4,HEARTS
3,6,DIAMONDS
4,JACK,HEARTS
0,KING,CLUBS


Figure out how do I match a straight. 

Get the position of the first value in the sorted dataframe in the card_values list.
Then compare the next value in both list to see if they match and so on. 

https://www.digitalocean.com/community/tutorials/how-to-compare-two-lists-in-python

In [56]:
first_value = df.iloc[0,0]
first_value

'3'

In [57]:
# Get position of first_value in card_values
# https://www.geeksforgeeks.org/python-list-index/

index_in_card_values = card_values.index(first_value)
index_in_card_values

2

In [ ]:
# Get the index of first_value in the card_values list.
# This is always going to match
if first_value == card_values[index_in_card_values]:
    print('Same value')

Same value


In [59]:
second_value = df.iloc[1,0]
second_value

'4'

In [61]:
third_value = df.iloc[2,0]
third_value

'6'

In [62]:
fourth_value = df.iloc[3,0]
fourth_value

'JACK'

In [63]:
fifth_value = df.iloc[4,0]
fifth_value

'KING'

In [ ]:
# Get value at the next index in card_values and compare with the next value in sorted df
if second_value == card_values[index_in_card_values +1]:
    print('Second match, Checking next value')
    if third_value == card_values[index_in_card_values +2]:
        if fourth_value == card_values[index_in_card_values +3]:
            if fifth_value == card_values[index_in_card_values +4]:
                print('You have a straight!!')
    else:
        print('You do not have a straight')

Second match, Checking next value
you do not have a straight


## References

[Geeksforgeeks.org, response.json() - Python requests](https://www.geeksforgeeks.org/response-json-python-requests/)

[W3 schools.com, Python Requests Module](https://www.w3schools.com/python/module_requests.asp)

[Realpython.com, Python's Requests Library (Guide)](https://realpython.com/python-requests/)

[Stackoverflow.com, How to access a dictionary key value present inside a list](https://stackoverflow.com/questions/6521892/how-to-access-a-dictionary-key-value-present-inside-a-list)

[Geeksforgeeks.org, Accessing nested dictionaries in a list](https://www.geeksforgeeks.org/accessing-value-inside-python-nested-dictionaries/) 

[Datacamp.com, importing json data and working with APIs](https://campus.datacamp.com/courses/streamlined-data-ingestion-with-pandas/importing-json-data-and-working-with-apis?ex=4)